In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling
from transformers import pipeline

In [ ]:
# Load GPT-2 model and tokenizer
model_name = "gpt2"  # You can also use "gpt2-medium", "gpt2-large", etc.
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
# Prepare the dataset
def load_dataset(train_file, tokenizer, block_size=128):
    dataset = TextDataset(
        tokenizer=tokenizer,
        file_path="/content/sample_data/train.txt",
        block_size=block_size,
    )
    return dataset

def load_data_collator(tokenizer):
    return DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False,
    )

# Define the file path to your dataset
train_file = "/content/sample_data/train.txt"  # Replace with your text file

# Load the dataset and data collator
train_dataset = load_dataset(train_file, tokenizer)
data_collator = load_data_collator(tokenizer)

In [ ]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",  # Output directory
    overwrite_output_dir=True,
    num_train_epochs=3,  # Number of training epochs
    per_device_train_batch_size=2,  # Batch size
    save_steps=10_000,  # Save checkpoint every 10,000 steps
    save_total_limit=2,  # Limit the total amount of checkpoints
    logging_steps=200,  # Log every 200 steps
)

In [ ]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

# Fine-tune the model
trainer.train()

# Save the model and tokenizer
trainer.save_model("./gpt2-finetuned") # Saves the model weights
tokenizer.save_pretrained("./gpt2-finetuned") # Saves the tokenizer

# Testing the fine-tuned model
generator = pipeline('text-generation', model='./gpt2-finetuned')
output = generator("The fine-tuned model says:", max_length=50)
print(output)

Step,Training Loss


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'The fine-tuned model says: "What you would expect is that your new car will have a \'pioneer\' engine. What I have noticed is that by using new technologies in the transmission and boost cables, it comes out to more'}]
